In [1]:
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# import win32file
# win32file._setmaxstdio(2048)

In [2]:
# Load data
# PER_data_folder = 'PER'
PER_data_folder = 'PER'
df = pd.read_csv(os.path.join(PER_data_folder, 'news_data_with_PER.csv'))

In [3]:
df.head()

,news,keywords,title,link,content,content_cn,PER
0,appledaily,人口販運,台灣捲入跨國人口販運風暴｜蘋果新聞網｜蘋果日報,http://www.appledaily.com.tw/appledaily/articl...,《報導者》獨家授權記者╱《報導者》蔣宜婷、李雪莉 台北─柬埔寨報導台灣已捲入遠洋漁業的人口販...,《报导者》独家授权记者╱《报导者》蒋宜婷、李雪莉 台北─柬埔寨报导台湾已卷入远洋渔业的人口贩...,"蒋宜婷,李雪莉"
1,appledaily,人口販運,​內政部：今年1到9月查緝人口販運被害人安置印尼人最多｜蘋果新聞網 ...,http://www.appledaily.com.tw/realtimenews/arti...,內政部公佈統計數據指出，今年1到9月司法及警察機關查緝人口販運案件共116件，75%為性剝削...,内政部公布统计数据指出，今年1到9月司法及警察机关查缉人口贩运案件共116件，75%为性剥削...,张文馨
2,appledaily,人口販運,打擊人口販運台成立防制人口販運國際工作坊| 蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,內政部「2018防制人口販運國際工作坊」今（25）日盛大揭幕，邀請副總統陳建仁、美國在台協會...,内政部「2018防制人口贩运国际工作坊」今（25）日盛大揭幕，邀请副总统陈建仁、美国在台协会...,"陈建仁,谷立言,陈建仁,谷立言,罗秉成,徐国勇,Jean,Patrick Taran,吴玉琴..."
3,appledaily,人口販運,赴美留學成奴隸！洛城破獲大宗人口販賣｜蘋果新聞網｜蘋果日報https://,https://www.appledaily.com.tw/realtimenews/art...,（新增配音影片）洛縣人口販賣執行組周二宣佈，搗破一宗全國性人口販賣案，3日的掃蕩行動中拘捕5...,（新增配音影片）洛县人口贩卖执行组周二宣布，捣破一宗全国性人口贩卖案，3日的扫荡行动中拘捕5...,"伍国庆,张紫茵,洛杉矶"
4,appledaily,人口販運,美人口販運報告台列最佳等級、中國最差｜蘋果新聞網｜蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,美國國務院昨日公布年度人口販運問題報告，台灣連續第9年名列最佳第一級國家。中國則被列為人口販...,美国国务院昨日公布年度人口贩运问题报告，台湾连续第9年名列最佳第一级国家。中国则被列为人口贩...,"史密斯,庞皮欧"


In [4]:
# Iterate through each news and store in dictionary
News_Dictionary = {}
Crime_All = set()
Person_All = set()

for index, row in df.iterrows():
    crime = row['keywords']
    Crime_All.add(crime)
    
    if type(row['PER']) == str:
        person_list = row['PER'].split(',')
    else:
        person_list = []
    
    """ ckipner 
    
    ckipner = row['PER_ckip']
    if type(ckipner) == str:
        person_list = ckipner[ckipner.find("[")+2: ckipner.find("]")-1].split("', '")
    else:
        person_list = []
    """
    
    Person_All = Person_All.union(set(person_list))
    
    Crime_Person_list = person_list + [crime]
    News_Dictionary[str(index)] = list(set(Crime_Person_list))
    
# Create list with all crimes and people  
Crime_Person_All = Crime_All.union(Person_All)
Crime_Person_All = list(Crime_Person_All)    

print('News_Dictionary:', News_Dictionary, '\n')
print('Crime_Person_All:', Crime_Person_All)

News_Dictionary: {'0': ['人口販運', '李雪莉', '蒋宜婷'], '1': ['人口販運', '张文馨'], '2': ['吴玉琴', '徐国勇', '李丽芬', '人口販運', '罗秉成', '陈建仁', 'Patrick Taran', '刘文渊', '蔡', '谷立言', 'Jean'], '3': ['人口販運', '伍国庆', '洛杉矶', '张紫茵'], '4': ['人口販運', '史密斯', '庞皮欧'], '5': ['人口販運', '陈禹先'], '6': ['人口販運', '徐珮君'], '7': ['人口販運', '傅德恩', '徐珮君', '叶俊荣', '陈建仁'], '8': ['人口販運', '陈培煌'], '9': ['陈培煌', '人口販運', '川普', '依凡卡', '李丽华', '提勒森', '蔡英文'], '10': ['人口販運'], '11': ['人口販運', '徐珮君', '【韩政燕'], '12': ['人口販運'], '13': ['陈培', '人口販運', '苏珊', '柯佩吉', '李丽华', '雷克斯‧提勒森'], '14': ['人口販運', '「福甡', '福甡(音', '陈怡'], '15': ['人口販運'], '16': ['人口販運', '洛杉矶', '罗伊斯', '李德扬', '陈志豪', '\u200b\u200b)', '卡特兰'], '17': ['人口販運', '提勒森', '\xa0【', '周一（'], '18': ['人口販運', '李丽华', '仲介', '苏惠昭'], '19': ['......', '人口販運', '傅德恩', '叶俊荣', '陈建仁', 'Robert Forden'], '20': ['吴敦义', '陈纯敬', '人口販運', '坊」', '李定宇', '敦义', '莫天虎', '吴'], '21': ['人口販運', '张荣兴', '杨文科'], '22': ['人口販運', '格雷斯', '霍马茨', '菲茨帕特里克'], '23': ['人口販運', '阿蓝云', '阿蓝因', '李定宇'], '24': ['人口販運'], '25': ['人口販運'], '26': ['人口販運'], '27': ['人口販運', 

In [5]:
# Set base df_co (co-occurance matrix)
df_co = pd.DataFrame(columns = Crime_Person_All, index = Crime_Person_All)
df_co[:] = 0

news_amount = len(News_Dictionary.keys())
print('news_amount', news_amount)

# Iterate through each news and add one for each Crime_Person-Crime_Person relationship
#-> In this case, relationship equates to appearing in the same news
n = 0
for value in News_Dictionary.values():
    for item1 in value:
        for item2 in value:
            df_co[item1][item2] += 1
            df_co[item2][item1] += 1
    
    print('Finish: ', n)
    n += 1

df_co.head(20)

news_amount 7482
Finish:  0
Finish:  1
Finish:  2
Finish:  3
Finish:  4
Finish:  5
Finish:  6
Finish:  7
Finish:  8
Finish:  9
Finish:  10
Finish:  11
Finish:  12
Finish:  13
Finish:  14
Finish:  15
Finish:  16
Finish:  17
Finish:  18
Finish:  19
Finish:  20
Finish:  21
Finish:  22
Finish:  23
Finish:  24
Finish:  25
Finish:  26
Finish:  27
Finish:  28
Finish:  29
Finish:  30
Finish:  31
Finish:  32
Finish:  33
Finish:  34
Finish:  35
Finish:  36
Finish:  37
Finish:  38
Finish:  39
Finish:  40
Finish:  41
Finish:  42
Finish:  43
Finish:  44
Finish:  45
Finish:  46
Finish:  47
Finish:  48
Finish:  49
Finish:  50
Finish:  51
Finish:  52
Finish:  53
Finish:  54
Finish:  55
Finish:  56
Finish:  57
Finish:  58
Finish:  59
Finish:  60
Finish:  61
Finish:  62
Finish:  63
Finish:  64
Finish:  65
Finish:  66
Finish:  67
Finish:  68
Finish:  69
Finish:  70
Finish:  71
Finish:  72
Finish:  73
Finish:  74
Finish:  75
Finish:  76
Finish:  77
Finish:  78
Finish:  79
Finish:  80
Finish:  81
Finish:  

Finish:  657
Finish:  658
Finish:  659
Finish:  660
Finish:  661
Finish:  662
Finish:  663
Finish:  664
Finish:  665
Finish:  666
Finish:  667
Finish:  668
Finish:  669
Finish:  670
Finish:  671
Finish:  672
Finish:  673
Finish:  674
Finish:  675
Finish:  676
Finish:  677
Finish:  678
Finish:  679
Finish:  680
Finish:  681
Finish:  682
Finish:  683
Finish:  684
Finish:  685
Finish:  686
Finish:  687
Finish:  688
Finish:  689
Finish:  690
Finish:  691
Finish:  692
Finish:  693
Finish:  694
Finish:  695
Finish:  696
Finish:  697
Finish:  698
Finish:  699
Finish:  700
Finish:  701
Finish:  702
Finish:  703
Finish:  704
Finish:  705
Finish:  706
Finish:  707
Finish:  708
Finish:  709
Finish:  710
Finish:  711
Finish:  712
Finish:  713
Finish:  714
Finish:  715
Finish:  716
Finish:  717
Finish:  718
Finish:  719
Finish:  720
Finish:  721
Finish:  722
Finish:  723
Finish:  724
Finish:  725
Finish:  726
Finish:  727
Finish:  728
Finish:  729
Finish:  730
Finish:  731
Finish:  732
Finish:  733

Finish:  1278
Finish:  1279
Finish:  1280
Finish:  1281
Finish:  1282
Finish:  1283
Finish:  1284
Finish:  1285
Finish:  1286
Finish:  1287
Finish:  1288
Finish:  1289
Finish:  1290
Finish:  1291
Finish:  1292
Finish:  1293
Finish:  1294
Finish:  1295
Finish:  1296
Finish:  1297
Finish:  1298
Finish:  1299
Finish:  1300
Finish:  1301
Finish:  1302
Finish:  1303
Finish:  1304
Finish:  1305
Finish:  1306
Finish:  1307
Finish:  1308
Finish:  1309
Finish:  1310
Finish:  1311
Finish:  1312
Finish:  1313
Finish:  1314
Finish:  1315
Finish:  1316
Finish:  1317
Finish:  1318
Finish:  1319
Finish:  1320
Finish:  1321
Finish:  1322
Finish:  1323
Finish:  1324
Finish:  1325
Finish:  1326
Finish:  1327
Finish:  1328
Finish:  1329
Finish:  1330
Finish:  1331
Finish:  1332
Finish:  1333
Finish:  1334
Finish:  1335
Finish:  1336
Finish:  1337
Finish:  1338
Finish:  1339
Finish:  1340
Finish:  1341
Finish:  1342
Finish:  1343
Finish:  1344
Finish:  1345
Finish:  1346
Finish:  1347
Finish:  1348
Finish

Finish:  1888
Finish:  1889
Finish:  1890
Finish:  1891
Finish:  1892
Finish:  1893
Finish:  1894
Finish:  1895
Finish:  1896
Finish:  1897
Finish:  1898
Finish:  1899
Finish:  1900
Finish:  1901
Finish:  1902
Finish:  1903
Finish:  1904
Finish:  1905
Finish:  1906
Finish:  1907
Finish:  1908
Finish:  1909
Finish:  1910
Finish:  1911
Finish:  1912
Finish:  1913
Finish:  1914
Finish:  1915
Finish:  1916
Finish:  1917
Finish:  1918
Finish:  1919
Finish:  1920
Finish:  1921
Finish:  1922
Finish:  1923
Finish:  1924
Finish:  1925
Finish:  1926
Finish:  1927
Finish:  1928
Finish:  1929
Finish:  1930
Finish:  1931
Finish:  1932
Finish:  1933
Finish:  1934
Finish:  1935
Finish:  1936
Finish:  1937
Finish:  1938
Finish:  1939
Finish:  1940
Finish:  1941
Finish:  1942
Finish:  1943
Finish:  1944
Finish:  1945
Finish:  1946
Finish:  1947
Finish:  1948
Finish:  1949
Finish:  1950
Finish:  1951
Finish:  1952
Finish:  1953
Finish:  1954
Finish:  1955
Finish:  1956
Finish:  1957
Finish:  1958
Finish

Finish:  2532
Finish:  2533
Finish:  2534
Finish:  2535
Finish:  2536
Finish:  2537
Finish:  2538
Finish:  2539
Finish:  2540
Finish:  2541
Finish:  2542
Finish:  2543
Finish:  2544
Finish:  2545
Finish:  2546
Finish:  2547
Finish:  2548
Finish:  2549
Finish:  2550
Finish:  2551
Finish:  2552
Finish:  2553
Finish:  2554
Finish:  2555
Finish:  2556
Finish:  2557
Finish:  2558
Finish:  2559
Finish:  2560
Finish:  2561
Finish:  2562
Finish:  2563
Finish:  2564
Finish:  2565
Finish:  2566
Finish:  2567
Finish:  2568
Finish:  2569
Finish:  2570
Finish:  2571
Finish:  2572
Finish:  2573
Finish:  2574
Finish:  2575
Finish:  2576
Finish:  2577
Finish:  2578
Finish:  2579
Finish:  2580
Finish:  2581
Finish:  2582
Finish:  2583
Finish:  2584
Finish:  2585
Finish:  2586
Finish:  2587
Finish:  2588
Finish:  2589
Finish:  2590
Finish:  2591
Finish:  2592
Finish:  2593
Finish:  2594
Finish:  2595
Finish:  2596
Finish:  2597
Finish:  2598
Finish:  2599
Finish:  2600
Finish:  2601
Finish:  2602
Finish

...

Finish:  5517
Finish:  5518
Finish:  5519
Finish:  5520
Finish:  5521
Finish:  5522
Finish:  5523
Finish:  5524
Finish:  5525
Finish:  5526
Finish:  5527
Finish:  5528
Finish:  5529
Finish:  5530
Finish:  5531
Finish:  5532
Finish:  5533
Finish:  5534
Finish:  5535
Finish:  5536
Finish:  5537
Finish:  5538
Finish:  5539
Finish:  5540
Finish:  5541
Finish:  5542
Finish:  5543
Finish:  5544
Finish:  5545
Finish:  5546
Finish:  5547
Finish:  5548
Finish:  5549
Finish:  5550
Finish:  5551
Finish:  5552
Finish:  5553
Finish:  5554
Finish:  5555
Finish:  5556
Finish:  5557
Finish:  5558
Finish:  5559
Finish:  5560
Finish:  5561
Finish:  5562
Finish:  5563
Finish:  5564
Finish:  5565
Finish:  5566
Finish:  5567
Finish:  5568
Finish:  5569
Finish:  5570
Finish:  5571
Finish:  5572
Finish:  5573
Finish:  5574
Finish:  5575
Finish:  5576
Finish:  5577
Finish:  5578
Finish:  5579
Finish:  5580
Finish:  5581
Finish:  5582
Finish:  5583
Finish:  5584
Finish:  5585
Finish:  5586
Finish:  5587
Finish

Finish:  6162
Finish:  6163
Finish:  6164
Finish:  6165
Finish:  6166
Finish:  6167
Finish:  6168
Finish:  6169
Finish:  6170
Finish:  6171
Finish:  6172
Finish:  6173
Finish:  6174
Finish:  6175
Finish:  6176
Finish:  6177
Finish:  6178
Finish:  6179
Finish:  6180
Finish:  6181
Finish:  6182
Finish:  6183
Finish:  6184
Finish:  6185
Finish:  6186
Finish:  6187
Finish:  6188
Finish:  6189
Finish:  6190
Finish:  6191
Finish:  6192
Finish:  6193
Finish:  6194
Finish:  6195
Finish:  6196
Finish:  6197
Finish:  6198
Finish:  6199
Finish:  6200
Finish:  6201
Finish:  6202
Finish:  6203
Finish:  6204
Finish:  6205
Finish:  6206
Finish:  6207
Finish:  6208
Finish:  6209
Finish:  6210
Finish:  6211
Finish:  6212
Finish:  6213
Finish:  6214
Finish:  6215
Finish:  6216
Finish:  6217
Finish:  6218
Finish:  6219
Finish:  6220
Finish:  6221
Finish:  6222
Finish:  6223
Finish:  6224
Finish:  6225
Finish:  6226
Finish:  6227
Finish:  6228
Finish:  6229
Finish:  6230
Finish:  6231
Finish:  6232
Finish

Finish:  6763
Finish:  6764
Finish:  6765
Finish:  6766
Finish:  6767
Finish:  6768
Finish:  6769
Finish:  6770
Finish:  6771
Finish:  6772
Finish:  6773
Finish:  6774
Finish:  6775
Finish:  6776
Finish:  6777
Finish:  6778
Finish:  6779
Finish:  6780
Finish:  6781
Finish:  6782
Finish:  6783
Finish:  6784
Finish:  6785
Finish:  6786
Finish:  6787
Finish:  6788
Finish:  6789
Finish:  6790
Finish:  6791
Finish:  6792
Finish:  6793
Finish:  6794
Finish:  6795
Finish:  6796
Finish:  6797
Finish:  6798
Finish:  6799
Finish:  6800
Finish:  6801
Finish:  6802
Finish:  6803
Finish:  6804
Finish:  6805
Finish:  6806
Finish:  6807
Finish:  6808
Finish:  6809
Finish:  6810
Finish:  6811
Finish:  6812
Finish:  6813
Finish:  6814
Finish:  6815
Finish:  6816
Finish:  6817
Finish:  6818
Finish:  6819
Finish:  6820
Finish:  6821
Finish:  6822
Finish:  6823
Finish:  6824
Finish:  6825
Finish:  6826
Finish:  6827
Finish:  6828
Finish:  6829
Finish:  6830
Finish:  6831
Finish:  6832
Finish:  6833
Finish

Finish:  7374
Finish:  7375
Finish:  7376
Finish:  7377
Finish:  7378
Finish:  7379
Finish:  7380
Finish:  7381
Finish:  7382
Finish:  7383
Finish:  7384
Finish:  7385
Finish:  7386
Finish:  7387
Finish:  7388
Finish:  7389
Finish:  7390
Finish:  7391
Finish:  7392
Finish:  7393
Finish:  7394
Finish:  7395
Finish:  7396
Finish:  7397
Finish:  7398
Finish:  7399
Finish:  7400
Finish:  7401
Finish:  7402
Finish:  7403
Finish:  7404
Finish:  7405
Finish:  7406
Finish:  7407
Finish:  7408
Finish:  7409
Finish:  7410
Finish:  7411
Finish:  7412
Finish:  7413
Finish:  7414
Finish:  7415
Finish:  7416
Finish:  7417
Finish:  7418
Finish:  7419
Finish:  7420
Finish:  7421
Finish:  7422
Finish:  7423
Finish:  7424
Finish:  7425
Finish:  7426
Finish:  7427
Finish:  7428
Finish:  7429
Finish:  7430
Finish:  7431
Finish:  7432
Finish:  7433
Finish:  7434
Finish:  7435
Finish:  7436
Finish:  7437
Finish:  7438
Finish:  7439
Finish:  7440
Finish:  7441
Finish:  7442
Finish:  7443
Finish:  7444
Finish

,,肖民代,郑英庆,何荣幸,陈男受,李姓爱,何哲欣,林威君,吴提起,连福隆,...,刘乔安,谢震武,黄小珊,蔡佳燕,邹兴华,纳希尔,陈志勇,林承贤,吴良民,倪姓男子
,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
肖民代,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
郑英庆,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
何荣幸,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
陈男受,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
李姓爱,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
何哲欣,0,0,0,0,0,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
林威君,0,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
吴提起,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
连福隆,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Add weights to edges
items_amount = len(Crime_Person_All)
print('items_amount', items_amount)

n = 0
edge_list = []   # test networkx
for index, row in df_co.iterrows():
    i = 0
    for col in row:
        weight = float(col)     # Set weight - delete /464
        edge_list.append((index, df_co.columns[i], weight))
        i += 1
    print('Finish edge_list:', n)
    n += 1

# print('edge_list', edge_list, '\n')

items_amount 14409
Finish edge_list: 0
Finish edge_list: 1
Finish edge_list: 2
Finish edge_list: 3
Finish edge_list: 4
Finish edge_list: 5
Finish edge_list: 6
Finish edge_list: 7
Finish edge_list: 8
Finish edge_list: 9
Finish edge_list: 10
Finish edge_list: 11
Finish edge_list: 12
Finish edge_list: 13
Finish edge_list: 14
Finish edge_list: 15
Finish edge_list: 16
Finish edge_list: 17
Finish edge_list: 18
Finish edge_list: 19
Finish edge_list: 20
Finish edge_list: 21
Finish edge_list: 22
Finish edge_list: 23
Finish edge_list: 24
Finish edge_list: 25
Finish edge_list: 26
Finish edge_list: 27
Finish edge_list: 28
Finish edge_list: 29
Finish edge_list: 30
Finish edge_list: 31
Finish edge_list: 32
Finish edge_list: 33
Finish edge_list: 34
Finish edge_list: 35
Finish edge_list: 36
Finish edge_list: 37
Finish edge_list: 38
Finish edge_list: 39
Finish edge_list: 40
Finish edge_list: 41
Finish edge_list: 42
Finish edge_list: 43
Finish edge_list: 44
Finish edge_list: 45
Finish edge_list: 46
Fini

Finish edge_list: 383
Finish edge_list: 384
Finish edge_list: 385
Finish edge_list: 386
Finish edge_list: 387
Finish edge_list: 388
Finish edge_list: 389
Finish edge_list: 390
Finish edge_list: 391
Finish edge_list: 392
Finish edge_list: 393
Finish edge_list: 394
Finish edge_list: 395
Finish edge_list: 396
Finish edge_list: 397
Finish edge_list: 398
Finish edge_list: 399
Finish edge_list: 400
Finish edge_list: 401
Finish edge_list: 402
Finish edge_list: 403
Finish edge_list: 404
Finish edge_list: 405
Finish edge_list: 406
Finish edge_list: 407
Finish edge_list: 408
Finish edge_list: 409
Finish edge_list: 410
Finish edge_list: 411
Finish edge_list: 412
Finish edge_list: 413
Finish edge_list: 414
Finish edge_list: 415
Finish edge_list: 416
Finish edge_list: 417
Finish edge_list: 418
Finish edge_list: 419
Finish edge_list: 420
Finish edge_list: 421
Finish edge_list: 422
Finish edge_list: 423
Finish edge_list: 424
Finish edge_list: 425
Finish edge_list: 426
Finish edge_list: 427
Finish edg

Finish edge_list: 763
Finish edge_list: 764
Finish edge_list: 765
Finish edge_list: 766
Finish edge_list: 767
Finish edge_list: 768
Finish edge_list: 769
Finish edge_list: 770
Finish edge_list: 771
Finish edge_list: 772
Finish edge_list: 773
Finish edge_list: 774
Finish edge_list: 775
Finish edge_list: 776
Finish edge_list: 777
Finish edge_list: 778
Finish edge_list: 779
Finish edge_list: 780
Finish edge_list: 781
Finish edge_list: 782
Finish edge_list: 783
Finish edge_list: 784
Finish edge_list: 785
Finish edge_list: 786
Finish edge_list: 787
Finish edge_list: 788
Finish edge_list: 789
Finish edge_list: 790
Finish edge_list: 791
Finish edge_list: 792
Finish edge_list: 793
Finish edge_list: 794
Finish edge_list: 795
Finish edge_list: 796
Finish edge_list: 797
Finish edge_list: 798
Finish edge_list: 799
Finish edge_list: 800
Finish edge_list: 801
Finish edge_list: 802
Finish edge_list: 803
Finish edge_list: 804
Finish edge_list: 805
Finish edge_list: 806
Finish edge_list: 807
Finish edg

Finish edge_list: 1131
Finish edge_list: 1132
Finish edge_list: 1133
Finish edge_list: 1134
Finish edge_list: 1135
Finish edge_list: 1136
Finish edge_list: 1137
Finish edge_list: 1138
Finish edge_list: 1139
Finish edge_list: 1140
Finish edge_list: 1141
Finish edge_list: 1142
Finish edge_list: 1143
Finish edge_list: 1144
Finish edge_list: 1145
Finish edge_list: 1146
Finish edge_list: 1147
Finish edge_list: 1148
Finish edge_list: 1149
Finish edge_list: 1150
Finish edge_list: 1151
Finish edge_list: 1152
Finish edge_list: 1153
Finish edge_list: 1154
Finish edge_list: 1155
Finish edge_list: 1156
Finish edge_list: 1157
Finish edge_list: 1158
Finish edge_list: 1159
Finish edge_list: 1160
Finish edge_list: 1161
Finish edge_list: 1162
Finish edge_list: 1163
Finish edge_list: 1164
Finish edge_list: 1165
Finish edge_list: 1166
Finish edge_list: 1167
Finish edge_list: 1168
Finish edge_list: 1169
Finish edge_list: 1170
Finish edge_list: 1171
Finish edge_list: 1172
Finish edge_list: 1173
Finish edge

Finish edge_list: 1488
Finish edge_list: 1489
Finish edge_list: 1490
Finish edge_list: 1491
Finish edge_list: 1492
Finish edge_list: 1493
Finish edge_list: 1494
Finish edge_list: 1495
Finish edge_list: 1496
Finish edge_list: 1497
Finish edge_list: 1498
Finish edge_list: 1499
Finish edge_list: 1500
Finish edge_list: 1501
Finish edge_list: 1502
Finish edge_list: 1503
Finish edge_list: 1504
Finish edge_list: 1505
Finish edge_list: 1506
Finish edge_list: 1507
Finish edge_list: 1508
Finish edge_list: 1509
Finish edge_list: 1510
Finish edge_list: 1511
Finish edge_list: 1512
Finish edge_list: 1513
Finish edge_list: 1514
Finish edge_list: 1515
Finish edge_list: 1516
Finish edge_list: 1517
Finish edge_list: 1518
Finish edge_list: 1519
Finish edge_list: 1520
Finish edge_list: 1521
Finish edge_list: 1522
Finish edge_list: 1523
Finish edge_list: 1524
Finish edge_list: 1525
Finish edge_list: 1526
Finish edge_list: 1527
Finish edge_list: 1528
Finish edge_list: 1529
Finish edge_list: 1530
Finish edge

...

Finish edge_list: 10223
Finish edge_list: 10224
Finish edge_list: 10225
Finish edge_list: 10226
Finish edge_list: 10227
Finish edge_list: 10228
Finish edge_list: 10229
Finish edge_list: 10230
Finish edge_list: 10231
Finish edge_list: 10232
Finish edge_list: 10233
Finish edge_list: 10234
Finish edge_list: 10235
Finish edge_list: 10236
Finish edge_list: 10237
Finish edge_list: 10238
Finish edge_list: 10239
Finish edge_list: 10240
Finish edge_list: 10241
Finish edge_list: 10242
Finish edge_list: 10243
Finish edge_list: 10244
Finish edge_list: 10245
Finish edge_list: 10246
Finish edge_list: 10247
Finish edge_list: 10248
Finish edge_list: 10249
Finish edge_list: 10250
Finish edge_list: 10251
Finish edge_list: 10252
Finish edge_list: 10253
Finish edge_list: 10254
Finish edge_list: 10255
Finish edge_list: 10256
Finish edge_list: 10257
Finish edge_list: 10258
Finish edge_list: 10259
Finish edge_list: 10260
Finish edge_list: 10261
Finish edge_list: 10262
Finish edge_list: 10263
Finish edge_list

Finish edge_list: 10573
Finish edge_list: 10574
Finish edge_list: 10575
Finish edge_list: 10576
Finish edge_list: 10577
Finish edge_list: 10578
Finish edge_list: 10579
Finish edge_list: 10580
Finish edge_list: 10581
Finish edge_list: 10582
Finish edge_list: 10583
Finish edge_list: 10584
Finish edge_list: 10585
Finish edge_list: 10586
Finish edge_list: 10587
Finish edge_list: 10588
Finish edge_list: 10589
Finish edge_list: 10590
Finish edge_list: 10591
Finish edge_list: 10592
Finish edge_list: 10593
Finish edge_list: 10594
Finish edge_list: 10595
Finish edge_list: 10596
Finish edge_list: 10597
Finish edge_list: 10598
Finish edge_list: 10599
Finish edge_list: 10600
Finish edge_list: 10601
Finish edge_list: 10602
Finish edge_list: 10603
Finish edge_list: 10604
Finish edge_list: 10605
Finish edge_list: 10606
Finish edge_list: 10607
Finish edge_list: 10608
Finish edge_list: 10609
Finish edge_list: 10610
Finish edge_list: 10611
Finish edge_list: 10612
Finish edge_list: 10613
Finish edge_list

Finish edge_list: 10932
Finish edge_list: 10933
Finish edge_list: 10934
Finish edge_list: 10935
Finish edge_list: 10936
Finish edge_list: 10937
Finish edge_list: 10938
Finish edge_list: 10939
Finish edge_list: 10940
Finish edge_list: 10941
Finish edge_list: 10942
Finish edge_list: 10943
Finish edge_list: 10944
Finish edge_list: 10945
Finish edge_list: 10946
Finish edge_list: 10947
Finish edge_list: 10948
Finish edge_list: 10949
Finish edge_list: 10950
Finish edge_list: 10951
Finish edge_list: 10952
Finish edge_list: 10953
Finish edge_list: 10954
Finish edge_list: 10955
Finish edge_list: 10956
Finish edge_list: 10957
Finish edge_list: 10958
Finish edge_list: 10959
Finish edge_list: 10960
Finish edge_list: 10961
Finish edge_list: 10962
Finish edge_list: 10963
Finish edge_list: 10964
Finish edge_list: 10965
Finish edge_list: 10966
Finish edge_list: 10967
Finish edge_list: 10968
Finish edge_list: 10969
Finish edge_list: 10970
Finish edge_list: 10971
Finish edge_list: 10972
Finish edge_list

Finish edge_list: 11274
Finish edge_list: 11275
Finish edge_list: 11276
Finish edge_list: 11277
Finish edge_list: 11278
Finish edge_list: 11279
Finish edge_list: 11280
Finish edge_list: 11281
Finish edge_list: 11282
Finish edge_list: 11283
Finish edge_list: 11284
Finish edge_list: 11285
Finish edge_list: 11286
Finish edge_list: 11287
Finish edge_list: 11288
Finish edge_list: 11289
Finish edge_list: 11290
Finish edge_list: 11291
Finish edge_list: 11292
Finish edge_list: 11293
Finish edge_list: 11294
Finish edge_list: 11295
Finish edge_list: 11296
Finish edge_list: 11297
Finish edge_list: 11298
Finish edge_list: 11299
Finish edge_list: 11300
Finish edge_list: 11301
Finish edge_list: 11302
Finish edge_list: 11303
Finish edge_list: 11304
Finish edge_list: 11305
Finish edge_list: 11306
Finish edge_list: 11307
Finish edge_list: 11308
Finish edge_list: 11309
Finish edge_list: 11310
Finish edge_list: 11311
Finish edge_list: 11312
Finish edge_list: 11313
Finish edge_list: 11314
Finish edge_list

Finish edge_list: 11620
Finish edge_list: 11621
Finish edge_list: 11622
Finish edge_list: 11623
Finish edge_list: 11624
Finish edge_list: 11625
Finish edge_list: 11626
Finish edge_list: 11627
Finish edge_list: 11628
Finish edge_list: 11629
Finish edge_list: 11630
Finish edge_list: 11631
Finish edge_list: 11632
Finish edge_list: 11633
Finish edge_list: 11634
Finish edge_list: 11635
Finish edge_list: 11636
Finish edge_list: 11637
Finish edge_list: 11638
Finish edge_list: 11639
Finish edge_list: 11640
Finish edge_list: 11641
Finish edge_list: 11642
Finish edge_list: 11643
Finish edge_list: 11644
Finish edge_list: 11645
Finish edge_list: 11646
Finish edge_list: 11647
Finish edge_list: 11648
Finish edge_list: 11649
Finish edge_list: 11650
Finish edge_list: 11651
Finish edge_list: 11652
Finish edge_list: 11653
Finish edge_list: 11654
Finish edge_list: 11655
Finish edge_list: 11656
Finish edge_list: 11657
Finish edge_list: 11658
Finish edge_list: 11659
Finish edge_list: 11660
Finish edge_list

In [ ]:
# Remove edge if 0.0
updated_edge_list = [x for x in edge_list if not x[2] == 0.0]

# Create node for crime_or_person occurance in news
n = 0
node_list = []
for i in Crime_Person_All:
    for e in updated_edge_list:
        if i == e[0] and i == e[1]:
            node_list.append((i, e[2]))    # Set for the size of circles -  delete 6
    print('Finish node_list:', n)
    n += 1

In [ ]:
# Remove node if 0.0
n = 0    
max_occurrence = 0
for i in node_list:
    if i[1] == 0.0:
        node_list.remove(i)
    else:
        if i[1] > max_occurrence:
            max_occurrence = i[1]
    print('Finish node_list:', n)
    n += 1
        
# print('node_list', node_list)

In [ ]:
# Remove self references
n = 0
for i in updated_edge_list:
    if i[0] == i[1]:
        updated_edge_list.remove(i)
    print('Finish updated_edge_list:', n)
    n += 1

# print('updated_edge_list', updated_edge_list, '\n')

In [ ]:
# Networkx graph time!
import matplotlib.font_manager as fm
Fond_path = r'C:\Windows\Fonts\kaiu.ttf'
fp1 = fm.FontProperties(fname = Fond_path)
nx.set_fontproperties(fp1)
G = nx.Graph()
# nx.draw(G, with_labels=True)

n = 0
G = nx.Graph()
for i in sorted(node_list):
    G.add_node(i[0], size = i[1])
    print('Finish G.add_node:', n)
    n += 1
G.add_weighted_edges_from(updated_edge_list)

# Check data of graphs
# print('G.nodes', G.nodes(data = True), '\n')
# print('G.edges', G.edges(data = True))

In [ ]:
# Listing out the order of nodes for future work
node_order = list(nx.nodes(G))

# Reorder node list
n = 0
updated_node_list = []
for i in node_order:
    for x in node_list:
        if x[0] == i:
            updated_node_list.append(x)
    print('Finish updated_node_list:', n)
    n += 1

# print('updated_node_list:', updated_node_list, '\n')

In [ ]:
# Reorder edge list - this was a pain
test = nx.get_edge_attributes(G, 'weight')
n = 0
updated_again_edges = []
for i in nx.edges(G):
    for x in test.keys():
        if i[0] == x[0] and i[1] == x[1]:
            updated_again_edges.append(test[x])
    print('Finish updated_again_edges:', n)
    n += 1

# print('updated_again_edges:', updated_again_edges)

In [ ]:
# Set canvas size
plt.subplots(figsize=(12,12))

# Drawing custimization
node_scalar = 5000/max_occurrence       # Set node_scalar
edge_scalar = 10/max_occurrence        # Set edge_scalar
sizes = [x[1]*node_scalar for x in updated_node_list]
widths = [x*edge_scalar for x in updated_again_edges]

# Draw the graph
pos = nx.spring_layout(G, k=0.42, iterations=17)

nx.draw(G, pos, with_labels=True, font_size = 8, font_weight = 'bold', 
        node_size = sizes, width = widths)


# Save as png
output_file = os.path.join('imgs', 'co-occurrence_diagram.png')   
k = 1
while(1):
    if not os.path.exists(output_file): 
        plt.axis('off')
        plt.savefig(output_file)
        break
    else:
        k += 1
        output_file = os.path.join('imgs', 'co-occurrence_diagram_' + str(k) + '.png') 

In [ ]:
output_file = os.path.join('co-occurrence_csv', 'co-occurrence_matrix.csv')
k = 1
while(1):
    if not os.path.exists(output_file): 
        df_co.to_csv(output_file)
        break
    else:
        k += 1
        output_file = os.path.join('co-occurrence_csv', 'co-occurrence_matrix_' + str(k) + '.csv') 